In [ ]:
%load_ext ext.stackql

In [ ]:
## notebook variables
org_id = "12345"

In [ ]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

In [ ]:
## imports and object instantiation
import json, time, nest_asyncio, json
from pystackql import StackQL
import pandas as pd
from IPython.display import clear_output, display, Markdown, HTML
from ipytree import Tree, Node

stackql = StackQL()
nest_asyncio.apply()

In [ ]:
## functions
def display_cards(cards_data):
    cards_html = ''
    
    for title, value in cards_data:
        card_template = f"""
        <div style="
            border: 1px solid #e3e3e3;
            border-radius: 4px;
            padding: 20px;
            display: inline-block;
            margin: 5px;
            text-align: center;
            width: 150px;
            background-color: #f7f7f7;">
            <h4 style="margin: 5px 0;">{title}</h4>
            <span style="font-size: 30px; font-weight: bold; color: red;">{value}</span>
        </div>
        """
        cards_html += card_template
    
    display(HTML(cards_html))

def get_icon(resType):
    if resType == "project":
        return 'codepen'
    else:
        return resType

def print_overwrite(message):
    clear_output(wait=True)
    print(message)

def build_tree_node(df, parent_name, parent_node=None):
    children = df[df['parentDisplayName'] == parent_name]
    
    for _, child in children.iterrows():
        child_node = Node(child['displayName'], opened=False, icon=get_icon(child['resType']))
        if parent_node:
            parent_node.add_node(child_node)
        build_tree_node(df, child['displayName'], child_node)

def explode_json_list_col(input_df, col_to_explode, exploded_col):
    output_df = input_df
    output_df[col_to_explode] = output_df[col_to_explode].apply(json.loads)
    output_df = output_df.explode(col_to_explode).rename(columns={col_to_explode: exploded_col})
    return output_df

In [ ]:
#
# discover folders and projects
#
def get_projects_query(entity_id):
    return f"""
    SELECT displayName, name, parent, projectId 
    FROM google.cloudresourcemanager.projects 
    WHERE parent = '{entity_id}';
    """

def get_folders_query(entity_id):
    return f"""
    SELECT displayName, name, parent 
    FROM google.cloudresourcemanager.folders 
    WHERE parent = '{entity_id}';
    """

def get_resources_recursive(entity_id, parent_display_name='organization'):
    resources = []

    # Query for projects
    print_overwrite(f"Searching {entity_id} for projects...")
    project_query = get_projects_query(entity_id)
    project_results = json.loads(stackql.execute(project_query))

    if isinstance(project_results, list):
        print_overwrite(f"Found {len(project_results)} projects in {entity_id}")
        for proj in project_results:
            proj["parentDisplayName"] = parent_display_name
            proj["resType"] = "project"
            resources.append(proj)

    # Query for folders
    print_overwrite(f"Searching {entity_id} for folders...")
    folder_query = get_folders_query(entity_id)
    folder_results = json.loads(stackql.execute(folder_query))

    if isinstance(folder_results, list):
        print_overwrite(f"Found {len(folder_results)} folders in {entity_id}")
        for folder in folder_results:
            folder["parentDisplayName"] = parent_display_name
            folder["resType"] = "folder"
            resources.append(folder)

            # Fetch resources under this folder
            if 'name' in folder:
                resources.extend(get_resources_recursive(folder['name'], folder['displayName']))

    return resources

start_time = time.time()

# Start with the root organization to get all resources
all_resources = get_resources_recursive("organizations/%s" % (org_id))

# Convert list to dataframe and filter
resources_df = (pd.DataFrame(all_resources)
                .loc[lambda df: df['error'].isna()]
                .drop('error', axis=1, errors='ignore'))

# Create root node and build the tree
root = Node("organization", opened=False, icon='building')
build_tree_node(resources_df, "organization", root)

# Display the tree
tree = Tree(nodes=[root])

# Calculate metrics and display
elapsed_time = (time.time() - start_time) * 1000
num_folders = resources_df.query("resType == 'folder'").shape[0]
num_projects = resources_df.query("resType == 'project'").shape[0]
projects = resources_df.query("resType == 'project'")['projectId'].dropna().tolist()

print(f"Total elapsed time: {elapsed_time:.2f} ms")
cards_data = [("Number of Projects", num_projects), ("Number of Folders", num_folders)]
display_cards(cards_data)
tree

In [ ]:
#
# get project iam bindings
#
queries = [
    f"""
    SELECT role, condition, members, '{project}' as project
    FROM google.cloudresourcemanager.projects_iam_policies
    WHERE projectsId = '{project}'
    """
    for project in projects
]

res = stackql.executeQueriesAsync(queries)

bindings_df = (
    explode_json_list_col(pd.read_json(json.dumps(res)), 'members', 'member')
    .assign(**{
        'member_type': lambda x: x['member'].str.split(':', n=1).str[0],
        'member_email': lambda x: x['member'].str.split(':', n=1).str[1]
    })
    .drop('member', axis=1)
)

bindings_df

In [ ]:
#
# get zones and regions
#
queries = [
    f"""
    SELECT name, SPLIT_PART(region, '/', 9) as region
    FROM google.compute.zones
    WHERE project = '{project}'
    """
    for project in projects
]

all_results = []
for query in queries:
    res = stackql.execute(query)
    try:
        parsed_result = json.loads(res)
        all_results.extend(parsed_result)
    except json.JSONDecodeError:
        print(f"Failed to parse result from query: {query}")
        print(f"Raw result: {res}")
        
zones_df = (
    pd.DataFrame(all_results)
    .loc[lambda x: x['error'].isnull()]
    .drop('error', axis=1)
    .drop_duplicates()
)

regions = zones_df['region'].unique().tolist()
zones = zones_df['name'].tolist()